In [ ]:
import tkinter as tk
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
data = {
    'user_id': [1, 1, 1, 2, 2, 3, 3, 3, 4, 4],
    'movie_id': [1, 2, 3, 1, 3, 2, 3, 4, 2, 4],
    'rating': [5, 4, 3, 4, 5, 2, 4, 5, 3, 4]
}

df = pd.DataFrame(data)

# Create a user-item matrix
user_item_matrix = df.pivot_table(index='user_id', columns='movie_id', values='rating')
user_item_matrix = user_item_matrix.fillna(0)

# Calculate cosine similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def get_recommendations(user_id, num_recommendations=3):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]
    recommendations = []

    for similar_user in similar_users:
        similar_user_ratings = user_item_matrix.loc[similar_user]
        recommendations.extend(similar_user_ratings[similar_user_ratings > 0].index.tolist())

    recommendations = list(set(recommendations) - set(user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index))
    return recommendations[:num_recommendations]

def on_recommend():
    user_id = int(entry.get())
    recommendations = get_recommendations(user_id)
    result_text = f"Recommendations for User {user_id}: {', '.join(map(str, recommendations))}"
    result_label.config(text=result_text)

# Create the main window
window = tk.Tk()
window.title("Recommendation System")

# Create the entry box
entry_label = tk.Label(window, text="Enter User ID:")
entry_label.pack()
entry = tk.Entry(window, width=10)
entry.pack()

# Create the recommend button
recommend_button = tk.Button(window, text="Recommend", command=on_recommend)
recommend_button.pack()

# Create the result label
result_label = tk.Label(window, text="")
result_label.pack()

# Run the main loop
window.mainloop()
